# GCP Data Profiling & Talk to Your Data

This notebook:
1. **Auto-detects** your GCP data sources (GCS buckets, BigQuery datasets/tables) using Workbench credentials.
2. **Generates an automated data profiling report** (ydata-profiling) for the source you pick.
3. **Talk to your data**: you provide an LLM API key when you run that section; then ask any question about the loaded data.

Run cells in order. First run the setup and discovery cells, then pick a data source and load it.

## 1. Setup & install dependencies

In [ ]:
import sys
import subprocess
import os

# Install packages if not already present (run once)
try:
    import google.cloud.storage
    import ydata_profiling
    import openai
except ImportError:
    req = "/workspace/requirements.txt" if os.path.exists("/workspace/requirements.txt") else "/home/jovyan/requirements.txt"
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", req])
    print("Dependencies installed. Re-run this cell if needed.")
else:
    print("Dependencies OK.")

## 2. Authenticate with GCP & discover data sources

Uses **Application Default Credentials** (Workbench provides these). No API key needed for GCP.

In [ ]:
import gcp_tools as gt

# Get default GCP project (from Workbench / ADC)
GCP_PROJECT = gt.get_default_project()
if GCP_PROJECT:
    print(f"Default GCP project: {GCP_PROJECT}")
else:
    print("Could not detect default project. Set GCP_PROJECT manually below.")
    GCP_PROJECT = ""  # e.g. "your-project-id"

# Discover GCS buckets
buckets = gt.list_gcs_buckets(GCP_PROJECT)
print(f"GCS buckets found: {len(buckets)}")
if buckets:
    for b in buckets[:20]:
        print(f"  - {b}")
    if len(buckets) > 20:
        print(f"  ... and {len(buckets)-20} more")

# Discover BigQuery datasets (if project set)
bq_datasets = []
if GCP_PROJECT:
    try:
        bq_datasets = gt.list_bigquery_datasets(GCP_PROJECT)
        print(f"\nBigQuery datasets in {GCP_PROJECT}: {len(bq_datasets)}")
        for d in bq_datasets[:15]:
            print(f"  - {d}")
        if len(bq_datasets) > 15:
            print(f"  ... and {len(bq_datasets)-15} more")
    except Exception as e:
        print(f"BigQuery list error: {e}")

# Store for later cells
BUCKETS = buckets
BQ_DATASETS = bq_datasets

## 3. Pick a data source and load it

Set the variables below to your chosen source, then run the load cell.

- **GCS**: set `SOURCE_TYPE = 'GCS'`, `GCS_BUCKET`, `GCS_PATH`, and `GCS_FORMAT` (e.g. `'csv'`, `'parquet'`).
- **BigQuery**: set `SOURCE_TYPE = 'BigQuery'`, `BQ_PROJECT`, `BQ_DATASET`, `BQ_TABLE`. Optionally set `BQ_LIMIT` (default 100000).

In [ ]:
# --- Set your data source here ---
SOURCE_TYPE = "GCS"  # or "BigQuery"

# For GCS:
GCS_BUCKET = BUCKETS[0] if BUCKETS else ""  # or type bucket name
GCS_PATH = ""  # e.g. "path/to/file.csv"
GCS_FORMAT = "csv"  # csv, parquet, json

# For BigQuery:
BQ_PROJECT = GCP_PROJECT or ""
BQ_DATASET = BQ_DATASETS[0] if BQ_DATASETS else ""
BQ_TABLE = ""  # e.g. "mytable"
BQ_LIMIT = 100000

# Optional: list files in a bucket to choose one
if SOURCE_TYPE == "GCS" and GCS_BUCKET and not GCS_PATH:
    files = gt.list_gcs_blobs(GCS_BUCKET, max_results=100)
    print(f"Sample files in {GCS_BUCKET}: ")
    for f in files[:30]:
        print(f"  {f}")
    if len(files) > 30:
        print(f"  ... and {len(files)-30} more")
    print("\nSet GCS_PATH to one of the paths above and re-run the next cell.")

# Optional: list tables in a dataset to choose one
if SOURCE_TYPE == "BigQuery" and BQ_PROJECT and BQ_DATASET and not BQ_TABLE:
    tables = gt.list_bigquery_tables(BQ_PROJECT, BQ_DATASET)
    print(f"Tables in {BQ_PROJECT}.{BQ_DATASET}: {tables}")
    print("Set BQ_TABLE and re-run the next cell.")

In [ ]:
# Load data into DataFrame
if SOURCE_TYPE == "GCS" and GCS_BUCKET and GCS_PATH:
    df = gt.load_from_gcs(GCS_BUCKET, GCS_PATH, GCS_FORMAT)
    print(f"Loaded from gs://{GCS_BUCKET}/{GCS_PATH}: {len(df):,} rows, {len(df.columns)} columns")
elif SOURCE_TYPE == "BigQuery" and BQ_PROJECT and BQ_DATASET and BQ_TABLE:
    df = gt.load_from_bigquery(BQ_PROJECT, BQ_DATASET, BQ_TABLE, limit=BQ_LIMIT)
    print(f"Loaded from BigQuery {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}: {len(df):,} rows, {len(df.columns)} columns")
else:
    df = None
    print("Set SOURCE_TYPE and the required variables in the previous cell, then re-run.")

if df is not None:
    display(df.head(10))

## 4a. Automated data profiling report

Generates a **ydata-profiling** report: distributions, null %, min/max, and more for each column.

In [ ]:
if df is None:
    print("Load a data source above first.")
else:
    from ydata_profiling import ProfileReport

    profile = ProfileReport(
        df,
        title="Data Profiling Report",
        explorative=True,
        minimal=False,
    )
    profile.to_notebook_iframe()

In [ ]:
# Optional: save report to HTML file
if df is not None:
    profile.to_file("/workspace/data_profile_report.html")
    print("Report saved to /workspace/data_profile_report.html")

## 4b (optional). Get company OpenAI key from Secret Manager

If your company provides an OpenAI API key via **Google Cloud Secret Manager**, run the cell below first. Set:
- **project_id**: GCP project where the secret lives. For VWB Sandbox use `wb-smart-cabbage-5940`. For prod, use your workspace project (or get it from `wb status`).
- **team_alias**: Your team's secret prefix. The secret name is `{team_alias}openai-api-key`. Examples: `ml-platform-test-`, `compbio-`, `it-team-`, etc.

The key is fetched and stored in `LLM_API_KEY` and `OPENAI_BASE_URL` for the "Talk to your data" cells below. If you skip this, you can paste your own key when prompted in the next section.

In [ ]:
# --- Company key from Secret Manager (run this if you use it) ---
USE_SECRET_MANAGER = True  # Set to False to skip and paste your own key later

# VWB Sandbox (typical for dev). For prod, use your workspace project id.
SECRET_PROJECT_ID = "wb-smart-cabbage-5940"
# Your team's alias; secret name = {team_alias}openai-api-key
SECRET_TEAM_ALIAS = "ml-platform-test-"  # or compbio-, it-team-, participant-ops-, etc.

LLM_API_KEY = None
OPENAI_BASE_URL = "https://us.api.openai.com/v1/"  # Company endpoint

if USE_SECRET_MANAGER and SECRET_PROJECT_ID and SECRET_TEAM_ALIAS:
    try:
        LLM_API_KEY = gt.get_openai_key_from_secret_manager(SECRET_PROJECT_ID, SECRET_TEAM_ALIAS)
        print("Company OpenAI key loaded from Secret Manager.")
    except Exception as e:
        print(f"Secret Manager error: {e}")
        print("You can still paste your own key in the Talk to your data cell below.")

## 4c. Talk to your data

If you ran the Secret Manager cell above and the key was loaded, it will be used automatically. Otherwise you will be prompted to **paste your LLM API key** (not stored). Then ask any question about the loaded data.

In [ ]:
if df is None:
    print("Load a data source above first.")
else:
    from getpass import getpass

    if not (LLM_API_KEY and str(LLM_API_KEY).strip()):
        LLM_API_KEY = getpass("Paste your LLM API key (e.g. OpenAI): ")
    openai_base = globals().get("OPENAI_BASE_URL") or "https://api.openai.com/v1"
    data_summary, schema_and_sample = gt.data_summary_for_llm(df)
    question = input("Ask a question about the data: ")
    answer = gt.talk_to_data(
        api_key=LLM_API_KEY,
        data_summary=data_summary,
        schema_and_sample=schema_and_sample,
        question=question,
        model="gpt-4o",  # or gpt-4o-mini
        base_url=openai_base,
    )
    print("\n--- Answer ---\n")
    print(answer)

### Ask more questions (re-use same key)

Run the cell below to ask follow-up questions. Paste your API key again when prompted (or leave blank to skip).

In [ ]:
# Run this cell multiple times to ask more questions about the same df
if df is None:
    print("Load a data source above first.")
else:
    from getpass import getpass

    key = LLM_API_KEY if globals().get("LLM_API_KEY") else getpass("LLM API key (or press Enter to skip): ")
    if key:
        data_summary, schema_and_sample = gt.data_summary_for_llm(df)
        q = input("Question: ")
        if q:
            base = globals().get("OPENAI_BASE_URL") or "https://us.api.openai.com/v1/"
            print(gt.talk_to_data(key, data_summary, schema_and_sample, q, base_url=base))